이 자료는 위키독스 딥 러닝을 이용한 자연어 처리 입문의 허깅페이스 토크나이저 학습 자료입니다.  

링크 : https://wikidocs.net/99893

## **13-04 허깅페이스 토크나이저(Huggingface Tokenizer)**

자연어 처리 스타트업 허깅페이스가 개발한 패키지 tokenizers는 자주 등장하는 서브워드들을 하나의 토큰으로 취급하는 다양한 서브워드 토크나이저를 제공합니다. 이번 실습에서는 이 중에서 WordPiece Tokenizer를 실습해보겠습니다. 실습을 위해 우선 tokenizers를 설치합니다.

In [1]:
pip install tokenizers

Note: you may need to restart the kernel to use updated packages.


### **1. BERT의 워드피스 토크나이저(BertWordPieceTokenizer)**

구글이 공개한 딥 러닝 모델 BERT에는 WordPiece Tokenizer가 사용되었습니다. 허깅페이스는 해당 토크나이저를 직접 구현하여 tokenizers라는 패키지를 통해 버트워드피스토크나이저(BertWordPieceTokenizer)를 제공합니다.

여기서는 네이버 영화 리뷰 데이터를 해당 토크나이저에 학습시키고, 이로부터 서브워드의 단어 집합(Vocabulary)을 얻습니다. 그리고 임의의 문장에 대해서 학습된 토크나이저를 사용하여 토큰화를 진행합니다. 우선 네이버 영화 리뷰 데이터를 로드합니다.

In [2]:
import tokenizers
tokenizers.__version__

'0.13.1'

In [3]:
import pandas as pd
import urllib.request
from tokenizers import BertWordPieceTokenizer

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x272bfad6070>)

In [5]:
naver_df = pd.read_table('ratings.txt')
# naver_df = pd.read_csv('ratings.txt', delimiter='\t')
naver_df

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
199995,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
199996,3302770,쓰.레.기,0
199997,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
199998,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


센텐스피스(SentencePiece) 실습에서 진행했던 전처리와 동일한 과정을 진행합니다. ratings.txt라는 파일을 데이터프레임으로 로드한 후, 결측값을 제거하고, 실질적인 리뷰 데이터인 document열에 대해서 naver_review.txt라는 파일로 저장합니다.

In [8]:
naver_df.isnull().value_counts()

id     document  label
False  False     False    199992
       True      False         8
dtype: int64

In [9]:
naver_df = naver_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(naver_df.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [10]:
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

버트워드피스토크나이저를 설정합니다.

In [11]:
tokenizer = BertWordPieceTokenizer(lowercase=False, strip_accents=False)

각 인자가 의미하는 바는 다음과 같습니다.

* __lowercase__ : True일 경우 토크나이저는 영어의 대문자와 소문자를 동일한 문자 취급.  
* __strip_accents__ : True일 경우 악센트 제거.  
     ex) é → e, ô → o  
* __wordpieces_prefix__ : 서브워드로 쪼개졌을 경우 뒤의 서브워드에는 ##를 부착하여 원래 단어에서 분리된 것임을 표시.  
     ex) 안녕하세요 -> [안녕, ##하세요]  

네이버 영화 리뷰 데이터를 학습하여 단어 집합을 얻어봅시다.

In [12]:
data_file = 'naver_review.txt'
vocab_size = 30000
limit_alphabet = 6000
min_frequency = 5

tokenizer.train(files=data_file,
                vocab_size=vocab_size,
                limit_alphabet=limit_alphabet,
                min_frequency=min_frequency,
                )

각 인자가 의미하는 바는 다음과 같습니다.

* files : 단어 집합을 얻기 위해 학습할 데이터
* vocab_size : 단어 집합의 크기
* limit_alphabet : 병합 전의 초기 토큰의 허용 개수.  
* min_frequency : 최소 해당 횟수만큼 등장한 쌍(pair)의 경우에만 병합 대상이 된다.  
  
학습이 다 되었다면 vocab을 저장합니다. 경로를 지정해주어야 하는데 여기서는 현재 경로에 저장하겠습니다.

In [13]:
# vocab 저장
tokenizer.save_model('./')

['./vocab.txt']

vocab을 데이터프레임으로 로드합니다.

In [14]:
# vocab 로드  (fwf : fixed width formatted lines)
df = pd.read_fwf('vocab.txt', header=None)
df

,0
0,[PAD]
1,[UNK]
2,[CLS]
3,[SEP]
4,[MASK]
...,...
29995,말안
29996,말들이
29997,말라는
29998,말밖에는


총 30,000개의 단어가 존재합니다. 이는 단어 집합의 크기를 30,000으로 지정하였기 때문입니다. 실제 토큰화를 수행해봅시다.

In [15]:
encoded = tokenizer.encode('아 배고픈데 짜장면먹고싶다')
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',tokenizer.decode(encoded.ids))

토큰화 결과 : ['아', '배고', '##픈', '##데', '짜장면', '##먹고', '##싶다']
정수 인코딩 : [2111, 20632, 3988, 3279, 24682, 7871, 7379]
디코딩 : 아 배고픈데 짜장면먹고싶다


.ids는 실질적인 딥 러닝 모델의 입력으로 사용되는 정수 인코딩 결과를 출력합니다. tokens는 해당 토크나이저가 어떻게 토큰화를 진행했는지를 보여줍니다. decode()는 정수 시퀀스를 문자열로 복원합니다.

In [16]:
encoded = tokenizer.encode('커피 한잔의 여유를 즐기다')
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',tokenizer.decode(encoded.ids))

토큰화 결과 : ['커피', '한잔', '##의', '여유', '##를', '즐기', '##다']
정수 인코딩 : [12825, 25648, 3314, 12696, 3323, 10784, 3346]
디코딩 : 커피 한잔의 여유를 즐기다


### **2. 기타 토크나이저**

이 외 ByteLevelBPETokenizer, CharBPETokenizer, SentencePieceBPETokenizer 등이 존재하며 선택에 따라서 사용할 수 있습니다.

* __BertWordPieceTokenizer__ : BERT에서 사용된 워드피스 토크나이저(WordPiece Tokenizer)
* __CharBPETokenizer__ : 오리지널 BPE
* __ByteLevelBPETokenizer__ : BPE의 바이트 레벨 버전
* __SentencePieceBPETokenizer__ : 앞서 본 패키지 센텐스피스(SentencePiece)와 호환되는 BPE 구현체

In [17]:
from tokenizers import ByteLevelBPETokenizer, CharBPETokenizer, SentencePieceBPETokenizer

In [18]:
## SentencePieceBPETokenizer
tokenizer = SentencePieceBPETokenizer()
tokenizer.train('naver_review.txt', vocab_size=10000, min_frequency=5)

encoded = tokenizer.encode("이 영화는 정말 재미있습니다.")
print(encoded.tokens)

['▁이', '▁영화는', '▁정말', '▁재미있', '습니다.']


In [24]:
# ByteLevelBPETokenizer
bl_tokenizer = ByteLevelBPETokenizer()
# bl_tokenizer = ByteLevelBPETokenizer(unicode_normalizer="nfkc", trim_offsets=True)
bl_tokenizer.train('naver_review.txt', vocab_size=10000, min_frequency=5)

encoded = bl_tokenizer.encode("이 영화는 정말 재미있습니다.")
print(encoded.tokens)

['ìĿ´', 'ĠìĺģíĻĶëĬĶ', 'Ġìłķë§Ĳ', 'Ġìŀ¬ë¯¸ìŀĪìĬµëĭĪëĭ¤', '.']


In [20]:
# CharBPETokenizer
cb_tokenizer = CharBPETokenizer()
cb_tokenizer.train('naver_review.txt', vocab_size=10000, min_frequency=5)

encoded = cb_tokenizer.encode("이 영화는 정말 재미있습니다.")
print(encoded.tokens)

['이</w>', '영화는</w>', '정말</w>', '재미있습니다</w>', '.</w>']
